In [1]:
import sys
import os

sys.path.append(os.path.join(
    r'C:\Users\AA2-ControlPC\Software\NI-experiment-control\nistreamer'
))

In [2]:
from nistreamer import NIStreamer
from nistreamer.utils import iplot, RendOption
import numpy as np

In [3]:
ni_streamer = NIStreamer()

In [4]:
do_card = ni_streamer.add_do_card(max_name='PXI1Slot6', samp_rate=10e6)

do_bank = []
for port_idx in range(4):
    for line_idx in range(8):
        do_bank.append(do_card.add_chan(
            port_idx=port_idx,
            line_idx=line_idx
        ))

In [5]:
len(do_bank)

32

### Back-to-back instructions

In [6]:
ni_streamer.clear_edit_cache()

total_dur = 10.0
instr_dur = 10e-3
instr_num = int(total_dur // instr_dur)

t = 0
for idx in range(instr_num):
    
    print(f'Adding instruction {idx} ===============')
        
    for do_chan in do_bank:
        do_chan.high(t=t, dur=instr_dur)
        # do_chan.go_high(t=t)
        # do_chan.go_low(t=t + instr_dur - buf)
        
    t += instr_dur

Adding instruction 0 ===============
Adding instruction 1 ===============
Adding instruction 2 ===============
Adding instruction 3 ===============
Adding instruction 4 ===============
Adding instruction 5 ===============
Adding instruction 6 ===============
Adding instruction 7 ===============
Adding instruction 8 ===============
Adding instruction 9 ===============
Adding instruction 10 ===============


PanicException: Channel port0/line0
 Instruction InstrBook([CONST, {value: 1}], 999999-1099999, false) overlaps with the previous instruction InstrBook([CONST, {value: 1}], 900000-1000000, false)

Collision between

`90,000,0 ... 100,000,0`

and

`100,000,0-1 ... 110,000,0-1` (`99,999,9 ... 109,999,9`)

### Final instruction - collision with auto-calculated `stop_time`

In [7]:
ni_streamer.clear_edit_cache()

total_dur = 10.0
instr_dur = 10e-3
instr_num = int(total_dur // instr_dur)

# Add buffer to avoid back-to-back collision
buf = 50e-6

t = 0
for idx in range(instr_num):
        
    for do_chan in do_bank:
        do_chan.high(t=t, dur=instr_dur - buf)
        # do_chan.go_high(t=t)
        # do_chan.go_low(t=t + instr_dur - buf)
        
    t += instr_dur

In [8]:
ni_streamer.compile()

PanicException: Attempting to compile channel port1/line0 with stop_pos 99899498 while instructions end at 99899499

Previous test script version was reproducibly raising the same error but on a different channel 

```Python
ni_streamer.clear_edit_cache()

total_dur = 10.0
instr_dur = 10e-3
instr_num = int(total_dur // instr_dur)

# AO pulse settings
amp = 0.5
freq = 100e3
buf = 50e-6

t = 0
for idx in range(instr_num):
    
#     print(f'Adding instruction {idx} ===============')
    
    for ao_chan in ao_bank:
        ao_chan.sine(
            t=t, dur=instr_dur - buf,
            amp=amp, freq=freq
        )
        
    for do_chan in do_bank:
        do_chan.high(t=t, dur=instr_dur - buf)
        
    t += instr_dur
```

```
PanicException                            Traceback (most recent call last)
~\AppData\Local\Temp\ipykernel_2384\2904825007.py in <module>
----> 1 ni_streamer.compile()

~\Software\NI-experiment-control\niexpctrl\streamer.py in compile(self, stop_time)
     92     def compile(self, stop_time: Optional[float] = None) -> float:
     93         if stop_time is None:
---> 94             self._dll.compile()
     95         else:
     96             self._dll.compile_with_stoptime(stop_time=stop_time)

PanicException: Attempting to compile channel port2/line2 with stop_pos 99899498 while instructions end at 99899499
```

In [12]:
ni_streamer.compile()

PanicException: Attempting to compile channel port2/line2 with stop_pos 99899498 while instructions end at 99899499